# LangChain应用集成讯飞星火认知大模型

## 集成指南

讯飞星火认知大模型是国内领先的大模型，拥有跨领域的知识和语言理解能力，能够完成问答对话和文学创作等任务。星火大模型持续从海量文本数据和大规模语法知识中学习进化，实现从提出问题、规划问题到解决问题的全流程闭环。

星火大模型通过API提供访问。您可以在[https://xinghuo.xfyun.cn/sparkapi](https://xinghuo.xfyun.cn/sparkapi)申请API访问。

讯飞星火的**认知大模型**聊天补全（chat completion）利用了不同于OpenAI的机制，它通过WebSocket提供访问。本指南向您演示如何通过FastAPI来提供OpenAI接口协议兼容的星火认知大模型接口，通过此方式，我们能够在**LangChain**应用中使用**星火认知大模型**。

[spark-api-gateway](https://github.com/sugarforever/spark-api-gateway)正是本示例所使用的API网管，通过该网管提供与OpenAI接口协议兼容的API。spark-api-gateway目前支持如下聊天补全API：

- POST /v1/chat/completions

In [ ]:
! pip install langchain -q -U

## 示例1 - 简单的聊天补全

In [ ]:
from langchain.chat_models import ChatOpenAI

spark_chat = ChatOpenAI(openai_api_base="http://localhost:8000/v1", openai_api_key="sk-xxx")

In [ ]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(content="大语言模型的理论基础是什么？"),
]

In [ ]:
spark_chat.invoke(messages)

## 示例2 - LLMChain使用星火认知大模型

In [ ]:

from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
from langchain.chains import LLMChain
chat_llm_chain = LLMChain(
    llm=spark_chat,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
chat_llm_chain.predict(human_input="你是谁？")

In [ ]:
chat_llm_chain.predict(human_input="请解释第一性原理")

In [ ]:
chat_llm_chain.predict(human_input="这位哲学家的最大成就是什么？")

In [ ]:
chat_llm_chain.predict(human_input="他在心理学方面的贡献是什么？")

In [ ]:
chat_llm_chain.predict(human_input="哪里能买到这本书？")